# Multiclass SVM 구현

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y.unique())

['setosa' 'versicolor' 'virginica']


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [4]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [5]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [6]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [15]:
y_train

110     virginica
69     versicolor
148     virginica
39         setosa
53     versicolor
          ...    
64     versicolor
91     versicolor
81     versicolor
51     versicolor
0          setosa
Name: species, Length: 120, dtype: object

In [79]:
class OneVsRestSVM:
    def __init__(self, n_classes=3):
        self.n_classes = n_classes
        self.clfs = []
        self.y_pred = []
    
    # y를 onehot encoding하는 과정
    def onehot(self, y):
        #make categorical to integer
        onehot = np.zeros((y.shape[0], 3))

        yu= np.unique(y)
        for i in range(y.shape[0]):
            idx= np.where( yu==np.array(y)[i])
            onehot[i, idx[0][0]] = 1
        return onehot
    def one_vs_rest_labels(self, y_train):
        y_train = pd.get_dummies(y_train)
        return y_train
    
    # encoding된 y를 가져와서 class 개수 만큼의 classifier에 각각 돌리는 과정
    def fit(self, X_train, y_train, C=5, gamma=5):
        # y encoding
        y_encoded = self.onehot(y_train)
        
        for i in range(self.n_classes):
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
            clf.fit(X_train, y_encoded[:,i])
            self.clfs.append(clf)

    # 각각의 classifier에서 나온 결과를 바탕으로 투표를 진행하는 과정
    def predict(self, X_test):
        vote = np.zeros((len(X_test), 3), dtype=int)
        size = X_test.shape[0]
        
        # 각각의 classifier에서 나온 결과를 바탕으로 투표를 진행하는 과정
        for i in range(size):
            for j in range(self.n_classes):
                if self.clfs[j].predict(X_test)[i] == 1:
                    vote[i][j] += 1
                else:
                    vote[i][j] -= 1
    
            # 투표한 값 중 가장 큰 값의 인덱스를 test label에 넣는다
            self.y_pred.append(np.argmax(vote[i]))
            
            # 2번째, 가장 큰 인덱스가 여러개일 경우, decision_function의 값이 가장 큰 경우를 test label에 넣는다
            temp=np.array(vote[i])
            winner=np.argwhere(temp==np.amax(temp))[0]
            if len(winner)>1:
                cur_max=-1
                for j in range(len(winner)):
                    if self.clfs[winner[j]].decision_function(X_test)[i]>cur_max:
                        cur_max=self.clfs[winner[j]].decision_function(X_test)[i]
                        self.y_pred[i]=winner[j]       

        # test를 진행하기 위해 0,1,2로 되어있던 데이터를 다시 문자 label로 변환
        self.y_pred = pd.DataFrame(self.y_pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
        return self.y_pred

In [78]:
listy = [7, 6, 5, 7, 6, 7, 6, 6, 6, 4, 5, 6]
winner = np.argwhere(listy == np.amax(listy))
print(winner[:,0][0])

0


In [80]:
a=OneVsRestSVM()
a.fit(X_train, y_train)
y_pred = a.predict(X_test)

In [81]:
print(y_pred)

             0
0   versicolor
1   versicolor
2   versicolor
3       setosa
4    virginica
5    virginica
6       setosa
7    virginica
8       setosa
9   versicolor
10   virginica
11      setosa
12      setosa
13   virginica
14  versicolor
15  versicolor
16      setosa
17      setosa
18      setosa
19   virginica
20      setosa
21   virginica
22  versicolor
23  versicolor
24   virginica
25      setosa
26      setosa
27   virginica
28   virginica
29  versicolor
